In [3]:
import numpy as np
import pandas as pd
import time

from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, normalize

In [77]:
data = pd.read_csv('NY_rolling_sales_2018.csv')
data.dropna(inplace=True)
data = data[data['GROSS SQUARE FEET'] != 0]
data = data[data['SALE PRICE'] >= 100000]
data = data[data['YEAR BUILT'] != 0]

#data = data.reset_index()
data=data[data['BUILDING CLASS CATEGORY']=='01 ONE FAMILY DWELLINGS']

# Separate out the x_data and y_data.
data = data.loc[:, ['GROSS SQUARE FEET','YEAR BUILT','SALE PRICE','NEIGHBORHOOD']]


# The random state to use while splitting the data.
random_state = 100

In [243]:
crime_data = pd.read_csv('NY_neighborhoods_crime_index.csv')#'crime_index_per_neighborhood_data/crime_2017.csv'

In [244]:
crime_data.columns=['NEIGHBORHOOD', 'Crime_Index'] #[u'Neighborhood_ID', u'NEIGHBORHOOD', u'Crime_Rate', u'Year']

In [245]:
crime_data['NEIGHBORHOOD']=map(lambda x: x.upper(), crime_data['NEIGHBORHOOD'])

In [246]:
crime_data.head()

,NEIGHBORHOOD,Crime_Index
0,ALLERTON,NaN
1,ALLEY POND PARK,NaN
2,ARDEN HEIGHTS,3.488
3,ARLINGTON,NaN
4,ARROCHAR,3.971


In [247]:
df = pd.merge(data, crime_data, on='NEIGHBORHOOD', how='left')

In [253]:
df.dropna(inplace=True)
df.shape

(8567, 5)

In [258]:
df.to_csv('temp.csv')

In [271]:
df['UNIT PRICE'] = df['SALE PRICE']/x_data['GROSS SQUARE FEET']
x_data = df.loc[:, ['GROSS SQUARE FEET','YEAR BUILT','UNIT PRICE','Crime_Index']]
y_data = df.loc[:, "NEIGHBORHOOD"]
x_data.shape

(8567, 4)

In [272]:
# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, shuffle=True, random_state=random_state)

In [261]:
#scaler = StandardScaler()
#scaler.fit(x_train)
#x_train_scaled = scaler.transform(x_train)
#x_test_scaled = scaler.transform(x_test)

In [273]:
# Random forest
rf = RandomForestClassifier(random_state=random_state)
rf.fit(x_train,y_train)

train_prediction = rf.predict(x_train)
train_acc = accuracy_score(y_train, train_prediction)
print('Train Accuracy is: {0:.2f}'.format(train_acc))
test_prediction = rf.predict(x_test)
test_acc = accuracy_score(y_test, test_prediction)
print('Test Accuracy is: {0:.2f}'.format(test_acc))

Train Accuracy is: 1.00
Test Accuracy is: 0.90


In [274]:
res=rf.predict_proba(x_test)

In [275]:
recommendation=np.empty([res.shape[0],3],dtype="S30")
for i in range(res.shape[0]):
    top=np.argsort(res[i,])[::-1][:3]
    #recommendation=np.append(recommendation,[rf.classes_[top]],axis=0)
    recommendation[i,:]=rf.classes_[top]

In [276]:
recommendation.shape

(2571, 3)

In [277]:
y_test[:10]

3600               BAYSIDE
4280          FAR ROCKAWAY
10896         NEW BRIGHTON
1113        WILLIAMSBRIDGE
9528     CASTLETON CORNERS
5423                HOLLIS
2681           MARINE PARK
5737       JACKSON HEIGHTS
10760        MIDLAND BEACH
1657              CANARSIE
Name: NEIGHBORHOOD, dtype: object

In [278]:
eval=np.empty([res.shape[0],],dtype="bool")

In [279]:
for i in range(res.shape[0]):
    eval[i]=False
    if np.array(y_test)[i] in recommendation[i]:
        eval[i]=True

In [280]:
1.0*eval.sum()/res.shape[0]

0.9618825359782186